In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [3]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()


[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1758017519199, experiment_id='1', last_update_time=1758017519199, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1758016419729, experiment_id='0', last_update_time=1758016419729, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
client.create_experiment(name="my-cool-experiment")

'2'

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [9]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.20f}")

run id: 60438bf0d747426ab41d77964c9ae968, rmse: 6.30776211939925168792
run id: b4978c7c6c064ad1b22435c1e0a2b4d5, rmse: 6.31282807073208918069
run id: 8cfa290822424ce087dd943e612fc68d, rmse: 6.31358925738405307726
run id: 3ba5503743d5442baacca1f4f526632f, rmse: 6.31358925738405307726
run id: 486e071227f644fcbcf01ed923ee0779, rmse: 6.31358925738405307726


In [10]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [15]:
run_id = "60438bf0d747426ab41d77964c9ae968"
model_uri = f"runs:/{run_id}/models_mlflow"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-experiment")

Registered model 'nyc-taxi-experiment' already exists. Creating a new version of this model...


MlflowException: Unable to find a logged_model with artifact_path models_mlflow under run 60438bf0d747426ab41d77964c9ae968

In [16]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
artifacts = client.list_artifacts("60438bf0d747426ab41d77964c9ae968")
for a in artifacts:
    print(a.path)


In [18]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment_id = "1"  # hoặc experiment bạn đang dùng
runs = client.search_runs(experiment_ids=[experiment_id], max_results=100)
for r in runs:
    print(r.info.run_id, r.data.metrics)


8cfa290822424ce087dd943e612fc68d {'rmse': 6.313589257384053}
49e5a3d301c947b2b709c363209ac9ae {'rmse': 39.86140931095532}
3ba5503743d5442baacca1f4f526632f {'rmse': 6.313589257384053}
541bfd803e4a43ada859ff53aabb96b6 {'rmse': 12.143233939663894}
486e071227f644fcbcf01ed923ee0779 {'rmse': 6.313589257384053}
73b593389efd406d81f9d11379d3ad33 {'rmse': 12.143233939663894}
2058e0dff1884e6ca458e84b873de245 {}
e2fb330f7f4b48fcbaec54bedd2b4a62 {'rmse': 6.313589257384053}
bb84a5ec8bd24c3ea047784adb375680 {'rmse': 12.143233939663894}
0f2dd2fad4574d8aa68aeb2b6b915d01 {'rmse': 6.313589257384053}
45eb5a79db0942e99afe1e9f65c8beb3 {'rmse': 12.143233939663894}
18b3c820b6ed42d2a5f763a65f544a16 {'rmse': 6.313589257384053}
0c511f3dafa8462592234de237971793 {'rmse': 12.143233939663894}
303d82c248ac4558aa669dbafe98fa55 {}
23dac403502644e885367e5896c95d97 {'validation-rmse': 6.3135892542100205, 'stopped_iteration': 999.0, 'best_iteration': 999.0}
5c148b4a4cee4db78bae225c45c14658 {}
b4978c7c6c064ad1b22435c1e0a2b

In [20]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
run_id = "60438bf0d747426ab41d77964c9ae968"

artifacts = client.list_artifacts(run_id)

if not artifacts:
    print("Run này chưa log artifact nào.")
else:
    print("Artifacts trong run:")
    for a in artifacts:
        print(a.path)


Run này chưa log artifact nào.


In [21]:
from mlflow.tracking import MlflowClient
import mlflow

run_id = "26c948900d7c433aa68e5a69f897631a"
client = MlflowClient()

# Kiểm tra artifact
artifacts = client.list_artifacts(run_id)
if not artifacts:
    print("Run này chưa log model, cần log model trước khi register.")
else:
    print("Artifacts có trong run:", [a.path for a in artifacts])

    # Giả sử artifact path là "model"
    model_uri = f"runs:/{run_id}/model"

    # Register model
    mlflow.register_model(model_uri=model_uri, name="nyc-taxi-experiment")
    print(f"Model từ run {run_id} đã được register thành công!")


Registered model 'nyc-taxi-experiment' already exists. Creating a new version of this model...
2025/09/18 09:58:38 WARNING mlflow.tracking._model_registry.fluent: Run with id 26c948900d7c433aa68e5a69f897631a has no artifacts at artifact path 'model', registering model based on models:/m-dd51bbfebacb40bf873131e4cf01e9d7 instead


Artifacts có trong run: ['feature_importance_weight.json', 'feature_importance_weight.png']


Created version '1' of model 'nyc-taxi-experiment'.


Model từ run 26c948900d7c433aa68e5a69f897631a đã được register thành công!


In [24]:
import mlflow.sklearn
from sklearn.linear_model import LinearRegression


with mlflow.start_run(run_id="26c948900d7c433aa68e5a69f897631a"):
    mlflow.sklearn.log_model(lr, artifact_path="models_mlflow")  # log model


NameError: name 'lr' is not defined

In [25]:
run_id = "b8904012c84343b5bf8ee72aa8f0f402"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...


MlflowException: Run with id=b8904012c84343b5bf8ee72aa8f0f402 not found